Loading Libraries
---



In [ ]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 8.0 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
#load libraries 
# pip install yfinance
import pandas as pd
import math
import pandas_datareader as web
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import keras
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Activation,concatenate
from keras import optimizers
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from numpy.random import seed
from scipy.stats import pearsonr

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 2.9 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

# Loading Dataset

In [ ]:
#load dataset to calculate RSI
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Week 4 Discovery/ohlc_data.csv')

In [ ]:
tickers = df['ticker'].unique()

In [ ]:
#load log returns 
import yfinance as yf
import numpy as np
from functools import reduce
#df= get_log_return(tickers, "2001-01-02", "2020-12-31")
def get_log_return(list_of_tickers, start_date, end_date, interval="1d"): 
    stocks = list()
    for ticker in list_of_tickers:
        ticker_data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        data={ticker: np.log(ticker_data['Adj Close']) - np.log(ticker_data['Adj Close'].shift(1))}
        #data = {ticker: ticker_data['Adj Close']}
        ticker_log_return = pd.DataFrame(data=data)
        #ticker_return = pd.DataFrame(data=data)
        stocks.append(ticker_log_return)
        #stocks.append(ticker_return)
    combined = reduce(lambda df1,df2: pd.merge(df1,df2,on='Date'), stocks)
    combined['date'] = ticker_log_return.index
    return combined

df1= get_log_return(tickers, "2001-01-02", "2020-12-31")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
 def RSI(prices, n=14):
        deltas = np.diff(prices)
        seed = deltas[:n+1]
        up = seed[seed >= 0].sum()/n
        down = -seed[seed < 0].sum()/n
        rs = up/down
        rsi = np.zeros_like(prices)
        rsi[:n] = 100. - 100./(1.+rs)

        for i in range(n, len(prices)):
            delta = deltas[i-1]  # The diff is 1 shorter

            if delta > 0:
                upval = delta
                downval = 0.
            else:
                upval = 0.
                downval = -delta

            up = (up*(n-1) + upval)/n
            down = (down*(n-1) + downval)/n

            rs = up/down
            rsi[i] = 100. - 100./(1.+rs)

        return rsi

# Feature Selection

In [ ]:
#columns 
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'ticker'],
      dtype='object')

In [ ]:
#check for correlation between input variables 
seed(1)
pearsonr(df['Open'], df['High']) # 0.9998674775 strong positive correlation so used Open 
pearsonr(df['Open'], df['Low']) #0.9998443635796925 strong positive correlation so used Open
pearsonr(df['Open'], df['Close']) #0.9997452726273854 strong positive correlation so used Close
pearsonr(df['Adj Close'], df['Close'])  # 0.986858324615821 trong positive correlation so used Adj Close
pearsonr(df['Adj Close'], df['Volume']) #-0.1832547243718910 weak correlation so will include Volume

#since Adj Close price is used to calculate Log returns which will be the input variable, we will use Adj Close, RSI and Volumne as our final input variables

(-0.18325472437189103, 0.0)

# Model Parameters and Sub Methods

In [ ]:
#set parameters for model
params = {
    "BATCH_SIZE": 50,
    "EPOCHS": 10,
    "LR": 0.00010000,
    "TIME_STEPS": 60
    }

TIME_STEPS = params['TIME_STEPS']
BATCH_SIZE = params['BATCH_SIZE']

In [ ]:
def build_timeseries(mat, y_col_index):
    
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]

    x = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))

    print("Length of inputs", dim_0)

    for i in range(dim_0):
        x[i] = mat[i:TIME_STEPS+i]
        y[i] = mat[TIME_STEPS+i, y_col_index]

    print("length of time-series - inputs", x.shape)
    print("length of time-series - outputs", y.shape)


    return x, y

# Model

In [ ]:
def create_lstm_model(hp):
      
    lstm_model = Sequential()
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=50, max_value=100, step=TIME_STEPS)
    lstm_model.add(LSTM(units=hp_units, input_shape=(x_t.shape[1], x_t.shape[2]),
                       return_sequences=True,
                        kernel_initializer='random_uniform'))
    lstm_model.add(LSTM(60,activation='relu'))
    lstm_model.add(Dense(20,activation='relu'))
    lstm_model.add(Dropout(0.05))
    lstm_model.add(Dense(1, activation='sigmoid'))
        
    #compile the model
     # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values= [ 0.01, 0.05, 0.1])
    #[0.01, 0.05, 0.1] 
    lstm_model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate))
  
    return lstm_model

In [ ]:
ticker

'AMGN'

Loop Code

Dates to run:
<br/>
1. 2001-2015 (train) --> 2016 (test)
<br/>
2. 2001-2016 (train) --> 2017 (test)
<br/>
3. 2001-2017 (train) --> 2018 (test)
<br/>
4. 2001-2018 (train) --> 2019 (test)
<br/>
5. 2001-2018 (train) --> 2020 (test)

In [ ]:
mse = {}
# total_predicted_returns= pd.DataFrame()
for ticker in tickers: #run for all tickers
  #Preparing Data 
  df_ticker = df[df['ticker'] == ticker]
  df_ticker['RSI'] = RSI(df_ticker['Adj Close'])
  returns= df1[[ticker]].reset_index()
  returns['Date'] = returns['Date'].apply(lambda x : x.strftime('%Y-%m-%d'))
  df_ticker =   df_ticker[['RSI','Date','Volume']]
  returns.Date.astype('datetime64[ns]')
  df_ticker.Date.astype('datetime64[ns]')
  data = pd.merge(  df_ticker[['RSI','Date','Volume']],  returns , on='Date')[1:].rename(columns={ticker : 'Returns'})
  #Splitting Train Test Set
  data = data[data['Date'] <'2018-01-01']
  df_train = data[data['Date'] <'2017-01-01'].dropna()
  predicted_period = data[(data['Date'] >= '2017-01-01') & (data['Date'] < '2018-01-01')][['Date']]
  df_test =  data[len(data) - len(predicted_period) - 60:]
  train_cols = ["RSI","Returns","Volume"]
  x = df_train.loc[:,train_cols].values
  #Feature Scaling
  min_max_scaler = MinMaxScaler(feature_range = (0,1))
  x_train = min_max_scaler.fit_transform(x)
  x_test = min_max_scaler.transform(df_test.loc[:,train_cols])
  #build time_series based on 50 Time Step for training data 
  x_t, y_t = build_timeseries(x_train, 1)
  print("Training size", x_t.shape, y_t.shape)

  #build time_series based on 50 Time Step for test data 
  x_test_t, y_test_t = build_timeseries(x_test, 1)
  print("Test size", x_test_t.shape, y_test_t.shape)

  #validation set for keras tuner 
  x_left,x_val = train_test_split(x_t, test_size=0.2, shuffle=False)
  y_left, y_val, =  train_test_split(y_t, test_size=0.2, shuffle=False)

  #Keras Hyper Tuner
  tuner = kt.RandomSearch(
    create_lstm_model,
        objective='val_loss',
    max_trials=5)
  tuner.search(x_t, y_t, epochs=5, validation_data=(x_val,y_val))
  #tuner.search(x_t, y_t, epochs=5)
  #Best model summary 
  lstm_model  = tuner.get_best_models()[0]   
  print(ticker,lstm_model.summary()) #Optimal parameters information 

  #Callback to prevent too much optimization 
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  #Model Training
  history_lstm = lstm_model.fit(x_t, y_t, epochs=params["EPOCHS"], verbose=1, batch_size=BATCH_SIZE, callbacks=[callback],
                                shuffle=False)
  y_pred_lstm = lstm_model.predict(x_test_t, batch_size=BATCH_SIZE)
  y_pred_lstm = y_pred_lstm.flatten()

  #Results
  error_lstm = mean_squared_error(y_test_t, y_pred_lstm)
  mse[ticker] = error_lstm
  print(ticker,error_lstm)

  y_pred_lstm_org = (y_pred_lstm * min_max_scaler.data_range_[1]) + min_max_scaler.data_min_[1]   #Inverse Transform 
  predicted_returns = pd.Series(y_pred_lstm_org).to_frame(ticker)
  predicted_period = predicted_period.reset_index().drop(columns=['index'])
  result = predicted_returns.join(predicted_period)
  if  total_predicted_returns.empty:
      total_predicted_returns= result 
  else: 
      total_predicted_returns = pd.merge(result, total_predicted_returns, on='Date')




Length of inputs 3964
length of time-series - inputs (3964, 60, 3)
length of time-series - outputs (3964,)
Training size (3964, 60, 3) (3964,)
Length of inputs 251
length of time-series - inputs (251, 60, 3)
length of time-series - outputs (251,)
Test size (251, 60, 3) (251,)
INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10800     
                                                                 
 lstm_1 (LSTM)               (None, 60)                26640     
                                                                 
 dense (Dense)               (None, 20)                1220      
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 38,681
Trainable params: 38,681
Non-trainable params: 0
____________